In [32]:
import numpy as np
import os
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.options.display.max_columns=500
pd.options.display.max_rows=500
pd.options.display.max_colwidth=500
np.set_printoptions(precision=3)

%run ./run_xgb.py

In [2]:
kfold = KFold(n_splits=3)

train_params = {
    'num_boost_round': 100,
    'verbose_eval': 10,
    'early_stopping_rounds': 10
}

booster_params = {
    'objective': 'multi:softmax',
    'eval_metric': 'mlogloss',
    'num_class': 5,
    'nthread': -1,
    'eta': .1,
    'max_depth': 5
}

log_params = {
    'description': 'proba',
    'result_path': './result'
}

In [3]:
df = pd.read_csv('./train.csv')
train_cols = list(df.iloc[:, 13:20].columns)
label_col = 'price_doc'

df['label_binned'] = pd.cut(df[label_col], 5, labels=[0,1,2,3,5])

In [28]:
def my_acc(labels, preds):
    return accuracy_score(labels, stats.mode(preds, 0)[0].reshape((-1, 1)))

In [33]:
res = run_xgb(train_df=df.loc[:2000, :],
              val_df=df.loc[2000:4000, :],
             train_cols=train_cols,
             label_cols='label_binned',
             booster_params=booster_params,
             train_params=train_params,
             log_params=log_params,
             kfold=kfold,
             metric=my_acc)

./result/09-02-19
[0]	train-mlogloss:1.37675	val-mlogloss:1.37518
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 10 rounds.
[10]	train-mlogloss:0.44656	val-mlogloss:0.43657
[20]	train-mlogloss:0.19026	val-mlogloss:0.17836
[30]	train-mlogloss:0.10091	val-mlogloss:0.09026
[40]	train-mlogloss:0.06750	val-mlogloss:0.06032
[50]	train-mlogloss:0.05433	val-mlogloss:0.05027
[60]	train-mlogloss:0.04860	val-mlogloss:0.04725
[70]	train-mlogloss:0.04587	val-mlogloss:0.04669
[80]	train-mlogloss:0.04444	val-mlogloss:0.04701
Stopping. Best iteration:
[72]	train-mlogloss:0.04550	val-mlogloss:0.04665

[0]	train-mlogloss:1.37336	val-mlogloss:1.37668
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 10 rounds.
[10]	train-mlogloss:0.43165	val-mlogloss:0.45361
[20]	train-mlogloss:0.17235	val-mlogloss:0.20773
[30]	train-mlogloss:0.

In [27]:
accuracy_score(df.loc[2000:4000, 'label_binned'], stats.mode(res[1], 0)[0].reshape((-1, 1)))

0.9880059970014993